In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

import rasterio

In [ ]:
dataset_folder = '../datasets/valid_locations/' 
density_file = dataset_folder + 'nl-amsv-201001-7415-laz_m2dens.tif'
lights_file = dataset_folder + 'export_115530.csv'
lights_out_file = dataset_folder + 'export_115530_valid.csv'

### Get lights data

In [ ]:
df = pd.read_csv(lights_file, skiprows=1, sep=';', encoding = 'unicode_escape')

In [ ]:
df.head(2)

In [ ]:
df['X'] = df['X'].str.replace(',', '.', regex=False)
df['Y'] = df['Y'].str.replace(',', '.', regex=False)

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df['X'], df['Y']))

In [ ]:
gdf.shape

In [ ]:
gdf.tail(2)

### Get density data

In [ ]:
img = rasterio.open(density_file) 

In [ ]:
array = img.read()
n_bands = array.shape[0]
n_bands

In [ ]:
# Create two 2d arrays of the pixel X and Y coordinates
height = img.shape[0]
width = img.shape[1]
cols, rows = np.meshgrid(np.arange(width), np.arange(height))
xs, ys = rasterio.transform.xy(img.transform, rows, cols)
xcoords = np.array(xs)
ycoords = np.array(ys)
array = np.concatenate((array, xcoords[None,:,:], ycoords[None,:,:]))

In [ ]:
df_density = pd.DataFrame(array.reshape([n_bands+2,-1]).T, columns=[f"band_{i+1}" for i in range(n_bands)]+['x','y'])

In [ ]:
df_density.head(2)

In [ ]:
df_density['band_1'].mean()

In [ ]:
df_density.shape

In [ ]:
gdf_density = gpd.GeoDataFrame(df_density, geometry=gpd.points_from_xy(df_density['x'], df_density['y']))

In [ ]:
del df_density

### Add threshold for density

In [ ]:
density_threshold = 500

In [ ]:
gdf_density_selection = gdf_density[gdf_density['band_1'] > density_threshold]

In [ ]:
gdf_density_selection.shape

In [ ]:
gdf_density.shape

In [ ]:
del gdf_density

### Create one polygon

In [ ]:
# Go from points to squares
gdf_density_selection['buffer'] = gdf_density_selection['geometry'].buffer(0.5, cap_style=3)

In [ ]:
# Dissolve
gdf_density_selection_dis = gpd.GeoDataFrame(geometry = gpd.GeoSeries(gdf_density_selection["buffer"].unary_union))

In [ ]:
gdf_density_selection_dis

In [ ]:
import contextily as cx
import matplotlib.pyplot as plt

In [ ]:
CRS = 'epsg:28992'

In [ ]:
gdf_density_selection_dis = gdf_density_selection_dis.set_crs(CRS)

In [ ]:
#fig, ax = plt.subplots(figsize=(8,8))
#
#gdf_density_selection_dis.boundary.plot(ax=ax)
#gdf_density_selection_dis.plot(ax=ax, alpha=0.5)
#
## Background
#cx.add_basemap(ax=ax, source=cx.providers.Esri.WorldImagery, crs=CRS)

In [ ]:
#fig, ax = plt.subplots(figsize=(8,8))
#
#gdf_density.set_geometry('buffer').boundary.plot(ax=ax)
#gdf_density.set_geometry('buffer').plot(ax=ax, alpha=0.5)
#
## Background
#cx.add_basemap(ax=ax, source=cx.providers.Esri.WorldImagery, crs=CRS)

### Check if light is inside valid area

In [ ]:
gdf_density_selection_dis['PointCloudDekking'] = 1

In [ ]:
gdf_merge = gdf.sjoin(gdf_density_selection_dis, how='left')

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

#gdf_density_selection_dis.boundary.plot(ax=ax)
gdf_density_selection_dis.plot(ax=ax, alpha=0.5)

gdf_merge.plot(ax=ax, color='red')
gdf_merge.dropna(axis=0, subset='PointCloudDekking').plot(ax=ax, color='green')

# Background
cx.add_basemap(ax=ax, source=cx.providers.Esri.WorldImagery, crs=CRS)

### Prepare for storing

In [ ]:
gdf_merge['PointCloudDekking'].fillna(0, inplace=True)

In [ ]:
gdf_merge.drop(['geometry', 'index_right'], axis=1, inplace=True)

In [ ]:
gdf_merge['X'] = gdf_merge['X'].str.replace('.', ',', regex=False)
gdf_merge['Y'] = gdf_merge['Y'].str.replace('.', ',', regex=False)

In [ ]:
gdf_merge.tail(3)

### Store

In [ ]:
gdf_merge.to_csv(lights_out_file, sep=';')